Source : https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings.ipynb

In [1]:
from google.cloud import aiplatform,storage,bigquery
import os

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= "L:\\gcp-project-0523-628d01f95284.json"

In [3]:
PROJECT_ID = "gcp-project-0523"
REGION     = 'us-central1'
BUCKET     = 'gcp-project-0523-ann-bucket'
BUCKET_URI = 'gs://gcp-project-0523-ann-bucket'

In [4]:
aiplatform.init(project        = PROJECT_ID,
                location       = REGION,
                staging_bucket = BUCKET)

In [5]:
%%time
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

NUM_ROWS = 10000

QUERY = f"""
        SELECT distinct q.id, q.title, q.body, q.tags, a.body as answers, a.score 
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
              INNER JOIN 
              (SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers`  where Score>0 ORDER BY Score desc) AS a 
              ON q.id = a.parent_id 
        where q.tags like '%python%'
        LIMIT {NUM_ROWS};
        """

query_job = client.query(QUERY)
rows = query_job.result()


# Convert to a dataframe
df = rows.to_dataframe()

# Examine the data
df.head()

CPU times: total: 812 ms
Wall time: 26.1 s


,id,title,body,tags,answers,score
0,58594502,How can I create a multi line input in ipython?,<p>I am reading the book Python Data Science H...,input|ipython|multiline,"<h2>ctrl+o</h2>\n<p>If you type ctrl+o, you sh...",46
1,11309779,"Wtforms, add a class to a form dynamically",<p>is there a way i could send a form's (css) ...,python|css|flask|wtforms,<p>Alternatively you can add the class in your...,55
2,25386119,"Whats the difference between a OneToOne, ManyT...",<p>I'm having a little difficulty getting my h...,python|django|many-to-many|foreign-key-relatio...,"<p>Well, there's essentially two questions her...",261
3,21843714,Cygwin gcc issue - cannot find Python.h,<p>So I downloaded Cygwin to build Matplotlib ...,python|gcc|matplotlib|cygwin,<p>in cygwin make sure the python-devel packag...,52
4,2057045,"os.makedirs doesn't understand ""~"" in my path",<p>I have a little problem with <code>~</code>...,python|path,<p>The conversion of <code>~/some_dir</code> t...,84


In [6]:
# Extract the question ids and question text
ids = df.id.tolist()
# Verify the length
len(ids)

100000

In [7]:
questions = df.title.tolist()

In [8]:
type(questions)

list

In [111]:
df[df.id == '49620638']

,id,title,body,tags,answers,score


### Instantiate the text encoding model

Use the sentence-t5 encoder developed by Google for converting text to embeddings.
https://tfhub.dev/google/sentence-t5/st5-base/1

    The sentence-T5 family of models encode text into high-dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language processing tasks.

    Our model is built on top of T5 (i.e. the Text-To-Text Transfer Transformer). It is trained on a variety of data sources and initialized from pre-trained T5 models with different model sizes. The input is variable-length English text and the output is a 768-dimensional vector. The sentence-T5 base model employs a 12-layer transformer architecture as the T5 base model does.

In [13]:
pip install --user protobuf==3.20.*

  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.54.2 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
mlflow 2.1.1 requires pytz<2023, but you have pytz 2023.3 which is incompatible.
mlflow-skinny 2.2.1 requires pytz<2023, but you have pytz 2023.3 which is incompatible.
kfp 1.8.9 requires google-auth<2,>=1.6.1, but you have google-auth 2.16.1 which is incompatible.
kfp 1.8.9 requires jsonschema<4,>=3.0.1, but you have jsonschema 4.17.3 which is incompatible.
kfp 1.8.9 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
kfp 1.8.9 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
azureml-mlflow 1.50.0 requires azure-storage-blob<=12.13.0,>=12.5.0, but you have azure-storage-blob 12.16.0 which is incompatible.


In [9]:
pip install cryptography==38.0.4

     ---------------------------------------- 2.4/2.4 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 40.0.2
    Uninstalling cryptography-40.0.2:
      Successfully uninstalled cryptography-40.0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\visha\\AppData\\Roaming\\Python\\Python39\\site-packages\\~ryptography\\hazmat\\bindings\\_rust.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install tensorflow_hub

     -------------------------------------- 100.6/100.6 kB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow_text

  Using cached tensorflow_text-2.10.0-cp39-cp39-win_amd64.whl (5.0 MB)
  Using cached tensorflow-2.10.1-cp39-cp39-win_amd64.whl (455.9 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-serv

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\anaconda3\\Lib\\site-packages\\google\\protobuf\\internal\\_api_implementation.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install --upgrade tensorflow-estimator==2.6.0


     -------------------------------------- 462.9/462.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0


In [2]:
pip install tensorflow==2.8

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
# Registers the ops.
import tensorflow_text as text  # noqa: F401

hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"

encoder = hub.KerasLayer(hub_url)


#The sentence-T5 family of models encode text into high-dimensional vectors that can be used for text classification, 
#semantic similarity, clustering and other natural language processing tasks.

#### Defining an encoding function
    Define a function to be used later that will take sentences and convert them to embeddings.

In [22]:
from typing import List
import numpy as np
from tqdm.auto import tqdm

In [76]:
def encode_text_to_embedding(text_encoder: hub.KerasLayer,
                             sentences: List[str], 
                             batch_size: int = 100
                            ) -> np.ndarray:
    embeddings_list = []

    # Process data in chunks to prevent out-of-memory errors
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[i : i + batch_size]
        embeddings_list.append(text_encoder(tf.constant(batch)))

    return np.squeeze(np.column_stack(embeddings_list))

In [33]:
questions = df.title.tolist()[:1000]

In [34]:
len(questions)

1000

In [35]:
embeddings_list = []
embeddings_list.append(encoder(tf.constant(questions)))

In [36]:
embeddings_list

[[<tf.Tensor: shape=(1000, 768), dtype=float32, numpy=
  array([[-0.038572  ,  0.00226967,  0.02160059, ...,  0.02348958,
          -0.03167297, -0.01010165],
         [-0.03147488,  0.0228835 ,  0.00152248, ...,  0.01359614,
          -0.02500932,  0.00279736],
         [-0.00658658, -0.00926851,  0.01614841, ...,  0.01407363,
          -0.04354771, -0.00984584],
         ...,
         [ 0.01884109, -0.00919422,  0.02825229, ..., -0.05292677,
          -0.02134562, -0.00788735],
         [-0.03124767, -0.0074994 ,  0.02516945, ..., -0.02321175,
          -0.0531763 , -0.02600672],
         [-0.02241482, -0.0227832 ,  0.01016914, ..., -0.00840264,
          -0.06871971, -0.00611643]], dtype=float32)>]]

In [16]:
type(embeddings_list)

list

In [18]:
len(embeddings_list)

1

In [93]:
len(embeddings_list[0][0])

1000

In [56]:
len(embeddings_list[0][0][0])

768

In [ ]:
###High dimensional array demonstration

In [90]:
L1 = [[[[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
        [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3],
        [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]
     ]]]

In [94]:
len(L1[0][0])

4

In [80]:
len(L1[0][0][0])

20

In [81]:
np.column_stack(L1)

array([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]])

In [82]:
np.squeeze(np.column_stack(L1))

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

In [91]:
L2 = np.squeeze(np.column_stack(L1))
L2.shape

(4, 20)

In [92]:
L2.T

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]])

In [95]:
np.dot(L1[0],L2.T)

array([[[ 20,  40,  60,  80],
        [ 40,  80, 120, 160],
        [ 60, 120, 180, 240],
        [ 80, 160, 240, 320]]])

In [96]:
scores = np.dot(L1[0],L2.T)

In [24]:
lst = np.column_stack(embeddings_list)

In [27]:
lst.shape

(1, 100, 768)

In [28]:
lst2 = np.squeeze(np.column_stack(embeddings_list))

In [29]:
lst2.shape

(100, 768)

In [95]:
q_embeddings_list = np.squeeze(np.column_stack(embeddings_list))

In [96]:
type(q_embeddings_list)

numpy.ndarray

In [97]:
q_embeddings_list

array([[ 1.3169483e-02,  2.7340414e-02,  3.1550184e-02, ...,
         1.1246066e-02,  3.2783556e-03, -4.4349139e-03],
       [-1.9455349e-02, -4.0820893e-02,  2.3072425e-02, ...,
         1.8257158e-09, -3.6535934e-02, -1.0548860e-02],
       [-4.0678620e-02, -2.3141373e-02,  4.6255188e-03, ...,
        -4.3815430e-03, -2.7072171e-02, -1.1001379e-02],
       ...,
       [-3.0536965e-02, -2.4204301e-02, -2.8230718e-03, ...,
         2.2510935e-02, -4.2244155e-02, -4.7448813e-03],
       [-1.8881045e-02,  1.6586743e-02, -1.7042593e-03, ...,
         1.2922417e-02, -3.6516368e-02, -3.0779257e-02],
       [-1.1474572e-02, -4.7917962e-02,  5.6705484e-03, ...,
         2.7421325e-02, -5.3231172e-02, -2.3600943e-02]], dtype=float32)

In [98]:
q_embeddings_list.T

array([[ 1.3169483e-02, -1.9455349e-02, -4.0678620e-02, ...,
        -3.0536965e-02, -1.8881045e-02, -1.1474572e-02],
       [ 2.7340414e-02, -4.0820893e-02, -2.3141373e-02, ...,
        -2.4204301e-02,  1.6586743e-02, -4.7917962e-02],
       [ 3.1550184e-02,  2.3072425e-02,  4.6255188e-03, ...,
        -2.8230718e-03, -1.7042593e-03,  5.6705484e-03],
       ...,
       [ 1.1246066e-02,  1.8257158e-09, -4.3815430e-03, ...,
         2.2510935e-02,  1.2922417e-02,  2.7421325e-02],
       [ 3.2783556e-03, -3.6535934e-02, -2.7072171e-02, ...,
        -4.2244155e-02, -3.6516368e-02, -5.3231172e-02],
       [-4.4349139e-03, -1.0548860e-02, -1.1001379e-02, ...,
        -4.7448813e-03, -3.0779257e-02, -2.3600943e-02]], dtype=float32)

In [99]:
len(q_embeddings_list)

100

In [101]:
q_embeddings_list[0]

array([ 1.31694833e-02,  2.73404140e-02,  3.15501839e-02, -1.05054099e-02,
        3.08542792e-02,  2.79087778e-02, -3.88957299e-02,  4.59389947e-02,
       -9.89061221e-03, -2.60292590e-02,  2.66218744e-02, -5.24537191e-02,
        7.66599625e-02,  9.55881272e-03,  6.48852140e-02,  1.63385235e-02,
        6.36337558e-03, -4.97882478e-02,  4.29074094e-02,  2.86968462e-02,
        5.14005758e-02,  2.03971937e-02, -4.20202352e-02,  3.47979628e-02,
       -6.62175044e-02,  4.84398790e-02,  2.32892614e-02, -3.69203314e-02,
       -4.95770946e-02,  2.59939320e-02,  7.22241029e-02, -1.01076625e-02,
       -2.42736507e-02,  5.93533516e-02, -4.97294404e-02,  2.00368408e-02,
       -4.57295999e-02, -8.95374641e-03,  5.85630676e-03,  4.31426466e-02,
        1.21344598e-02, -7.45959505e-02, -5.19129597e-02,  3.55175957e-02,
       -6.92378134e-02, -1.64847784e-02,  1.38519090e-02,  6.45031556e-02,
       -3.46397460e-02, -8.40046704e-02, -2.77309287e-02, -3.08566857e-02,
       -3.21039557e-02, -

In [102]:
scores = np.dot(q_embeddings_list[0], q_embeddings_list.T)

In [103]:
scores

array([0.9999999 , 0.7415476 , 0.76925373, 0.7932436 , 0.7717062 ,
       0.7644955 , 0.7303977 , 0.73797417, 0.73679227, 0.73679227,
       0.80906034, 0.8003937 , 0.7127653 , 0.75992453, 0.75936395,
       0.681207  , 0.7526722 , 0.7587849 , 0.7518131 , 0.7584425 ,
       0.7605802 , 0.7848927 , 0.72944295, 0.68232334, 0.80818045,
       0.76117384, 0.7077024 , 0.74766093, 0.6962    , 0.7611209 ,
       0.7607341 , 0.7333749 , 0.7631705 , 0.781137  , 0.67962325,
       0.78591114, 0.76460326, 0.7966902 , 0.70715797, 0.79452276,
       0.72278345, 0.77229965, 0.85258263, 0.7569643 , 0.7035832 ,
       0.737812  , 0.7672106 , 0.7468906 , 0.7914982 , 0.7643939 ,
       0.6933259 , 0.74284065, 0.76135015, 0.78626955, 0.76135004,
       0.7272816 , 0.74509054, 0.7787844 , 0.7570572 , 0.7851807 ,
       0.7151699 , 0.7602813 , 0.7858405 , 0.72572577, 0.7135728 ,
       0.7225715 , 0.78206325, 0.8212311 , 0.7514897 , 0.76920724,
       0.7485944 , 0.74817514, 0.74371356, 0.6998775 , 0.83376

Test the encoding function

    Encode a subset of data and see if the embeddings and distance metrics make sense.

    According to the sentence-T5 research paper (https://arxiv.org/pdf/2108.08877.pdf), the similarity of embeddings is calculated using the dot-product.

In [77]:
# Encode 500 questions
questions = df.title.tolist()[:100]
question_embeddings = encode_text_to_embedding(
    text_encoder=encoder,
    sentences=questions
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [78]:
DIMENSIONS = len(question_embeddings[0])

DIMENSIONS

768

In [13]:
question_index = 0

print(f"Query question = {questions[question_index]}")
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")
     

Query question = Run a process and kill it if it doesn't end within one hour
	0: Run a process and kill it if it doesn't end within one hour: 0.9999998807907104
	1: Changing Process Name using Shell for nagios monitoring with check_procs: 0.8037509322166443
	2: PyQt application closes successfully, but process is not killed?: 0.7928211092948914
	3: Easy way to delete a shelve .dat file left behind by my Python program?: 0.7924448251724243
	4: Execute a Jupyter notebook with papermill and output a unique filename: 0.7731866836547852
	5: Tell Python to wait/pause a 'for' loop: 0.765079140663147
	6: Error in ipython console "Exception [WinError 995] The I/O operation has been aborted": 0.7648062109947205
	7: Optimizing WER (Word Error Rate) code?: 0.7567319869995117
	8: How do I detect the Python version at runtime?: 0.749366283416748
	9: What is the pythonic/idiomatic way of filtering the output of a generator expression?: 0.7476474046707153
	10: Windows - running .py directly vs running

Save the train split in JSONL format.

In [14]:
import tempfile

# Create temporary file to write embeddings to
embeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

embeddings_file.name

'C:\\Users\\visha\\AppData\\Local\\Temp\\tmpmhnu151v.json'

In [17]:
embeddings_file.name

'C:\\Users\\visha\\AppData\\Local\\Temp\\tmpmhnu151v.json'

In [20]:
import json

BATCH_SIZE = 100

#with open(embeddings_file.name, "a") as f:
embeddings_file_name='embeddings_file.json'
with open(embeddings_file_name, "w") as f:
    for i in tqdm(range(0, len(questions), BATCH_SIZE)):
        id_chunk = ids[i : i + BATCH_SIZE]

        question_chunk_embeddings = encode_text_to_embedding(text_encoder=encoder, 
                                                             sentences=questions[i : i + BATCH_SIZE]
                                                            )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk, question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
UNIQUE_FOLDER_NAME = "embeddings_folder_unique"
remote_folder = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {embeddings_file_name} {remote_folder}

Copying file://embeddings_file.json [Content-Type=application/json]...
/ [0 files][    0.0 B/  1.1 MiB]                                                
ServiceException: 401 Anonymous caller does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).


In [47]:
!gcloud config list

[accessibility]
screen_reader = False
[compute]
region = us-central1
zone = us-central1-c
[core]
account = 'vishalgcp0523@gmail.com'
disable_usage_reporting = false
project = gcp-project-0523



Your active configuration is: [default]


In [35]:
!gcloud config set account 'vishalgcp0523@gmail.com'

Updated property [core/account].
ERROR: (gcloud.config.set) The project property must be set to a valid project ID, not the project name [PROJECT_ID]
To set your project, run:

  $ gcloud config set project PROJECT_ID

or to unset it, run:

  $ gcloud config unset project


In [43]:
print(PROJECT_ID)
!gcloud config set project PROJECT_ID

gcp-project-0523


ERROR: (gcloud.config.set) The project property must be set to a valid project ID, not the project name [PROJECT_ID]
To set your project, run:

  $ gcloud config set project PROJECT_ID

or to unset it, run:

  $ gcloud config unset project


In [ ]:
'''
#Ran the project update command on command prompt
(base) PS C:\Users\visha>
(base) PS C:\Users\visha>
(base) PS C:\Users\visha>
(base) PS C:\Users\visha> gcloud config set project 'gcp-project-0523'
WARNING: You do not appear to have access to project [gcp-project-0523] or it does not exist.
Are you sure you wish to set property [core/project] to gcp-project-0523?

Do you want to continue (Y/n)?  Y

Updated property [core/project].
(base) PS C:\Users\visha>
(base) PS C:\Users\visha>
(base) PS C:\Users\visha>
(base) PS C:\Users\visha> gcloud config list
[accessibility]
screen_reader = False
[compute]
region = us-central1
zone = us-central1-c
[core]
account = 'vishalgcp0523@gmail.com'
disable_usage_reporting = false
project = gcp-project-0523

Your active configuration is: [default]
(base) PS C:\Users\visha>
'''

### Create Indexes

In [49]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "questions from stackoverflow"

In [50]:
print(remote_folder)
print(DIMENSIONS)

gs://gcp-project-0523-ann-bucket/embeddings_folder_unique/
768


In [51]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(display_name=DISPLAY_NAME,
                                                                    contents_delta_uri=remote_folder,
                                                                    dimensions=DIMENSIONS,
                                                                    approximate_neighbors_count=150,
                                                                    distance_measure_type="DOT_PRODUCT_DISTANCE",
                                                                    leaf_node_embedding_count=500,
                                                                    leaf_nodes_to_search_percent=80,
                                                                    description=DESCRIPTION,
                                                                   )

Creating MatchingEngineIndex


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex


Create MatchingEngineIndex backing LRO: projects/473197248954/locations/us-central1/indexes/966681827047636992/operations/9212652004642390016


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/473197248954/locations/us-central1/indexes/966681827047636992/operations/9212652004642390016


MatchingEngineIndex created. Resource name: projects/473197248954/locations/us-central1/indexes/966681827047636992


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/473197248954/locations/us-central1/indexes/966681827047636992


To use this MatchingEngineIndex in another session:


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:


index = aiplatform.MatchingEngineIndex('projects/473197248954/locations/us-central1/indexes/966681827047636992')


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/473197248954/locations/us-central1/indexes/966681827047636992')


In [52]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/473197248954/locations/us-central1/indexes/966681827047636992'

In [53]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(display_name=DISPLAY_NAME,
                                                                  description=DISPLAY_NAME,
                                                                  public_endpoint_enabled=True,
                                                                  #network=VPC_NETWORK_FULL,
                                                                 )

Creating MatchingEngineIndexEndpoint


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint


Create MatchingEngineIndexEndpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/4739451674756644864


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/4739451674756644864


MatchingEngineIndexEndpoint created. Resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


To use this MatchingEngineIndexEndpoint in another session:


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:


index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304')


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304')


In [54]:
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

In [55]:
my_index_endpoint = my_index_endpoint.deploy_index(index=tree_ah_index,
                                                   deployed_index_id=DEPLOYED_INDEX_ID
                                                  )



Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/4865552464323018752


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/4865552464323018752


TimeoutError: Operation did not complete within the designated timeout of 900 seconds.

In [59]:
#my_index_endpoint = my_index_endpoint('projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304')

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304')

In [65]:
my_index_endpoint.gca_resource

name: "projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304"
display_name: "stack_overflow"
description: "stack_overflow"
deployed_indexes {
  id: "deployed_index_id_unique"
  index: "projects/473197248954/locations/us-central1/indexes/966681827047636992"
  create_time {
    seconds: 1685294461
    nanos: 275893000
  }
  index_sync_time {
    seconds: 1685295731
    nanos: 103469000
  }
  automatic_resources {
    min_replica_count: 2
    max_replica_count: 2
  }
  deployment_group: "default"
}
etag: "AMEw9yPKzYMB4aWti7Eaubaze9t4cSiliRJor8w71nKsza4vnunola6ISeKjOLdCyPdM"
create_time {
  seconds: 1685294387
  nanos: 431764000
}
update_time {
  seconds: 1685294387
  nanos: 983013000
}
public_endpoint_domain_name: "2145421878.us-central1-473197248954.vdb.vertexai.goog"

In [60]:
test_embeddings = encode_text_to_embedding(text_encoder=encoder,
                                           sentences=["How do I install tensorflow with GPU support?"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
test_embeddings

array([-3.46920043e-02, -6.79826736e-02,  4.70267944e-02,  7.88554735e-03,
       -3.78703587e-02, -3.27263549e-02, -4.16305335e-03,  4.51407507e-02,
       -1.70209594e-02, -4.59558144e-02,  3.49068902e-02, -3.51876095e-02,
        7.97955692e-02, -1.50319990e-02,  6.50781542e-02,  3.16455401e-02,
        1.23584569e-02, -4.27518925e-03,  2.01913230e-02,  2.80608609e-02,
        3.12974416e-02,  1.53770046e-02, -3.46761718e-02,  2.38961838e-02,
       -4.36633639e-02,  3.24505456e-02,  5.36020733e-02,  1.21157384e-02,
       -2.31851898e-02,  1.74899139e-02,  6.93742782e-02, -4.13260534e-02,
       -1.86444726e-02,  4.57508117e-02, -2.05389801e-02,  1.13655291e-02,
       -4.48801853e-02, -6.17317632e-02, -3.50356214e-02,  4.99744229e-02,
       -6.11441722e-03, -4.73947413e-02, -3.82532515e-02,  4.20878232e-02,
       -3.89112085e-02, -3.85740474e-02,  3.27653885e-02,  6.00255728e-02,
       -7.06862332e-03, -2.63761580e-02, -4.46173511e-02, -4.22799438e-02,
       -4.73105833e-02, -

In [69]:
my_index_endpoint

resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304

In [71]:
# Test query
NUM_NEIGHBOURS = 20
response = my_index_endpoint.match( deployed_index_id = "deployed_index_id_unique",
                                    queries           = [test_embeddings.tolist()],
                                    num_neighbors     = NUM_NEIGHBOURS,
                                  )

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "DNS resolution failed for :10000: UNKNOWN: unparseable host:port: ':10000'"
	debug_error_string = "UNKNOWN:DNS resolution failed for :10000: UNKNOWN: unparseable host:port: ':10000' {created_time:"2023-05-28T17:49:09.211233993+00:00", grpc_status:14}"
>

In [ ]:
response

In [ ]:
neighbor_ids = [neighbor.id for neighbor in response[0]]
neighbor_distances = [neighbor.distance for neighbor in response[0]]

for match_index, neighbor in enumerate(response[0]):
    titles = df[df.id.astype(str) == neighbor.id].title.tolist()

    if len(titles) > 0:
        print(
            f"{match_index}: title = '{titles[0]}', distance = {neighbor.distance:0.2f}"
        )

In [ ]:
CLEANING UP

In [72]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/8072115399010811904


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304/operations/8072115399010811904


MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


...

INFO:google.cloud.aiplatform.base:Deleting MatchingEngineIndexEndpoint : projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


Delete MatchingEngineIndexEndpoint  backing LRO: projects/473197248954/locations/us-central1/operations/2627263449519882240


INFO:google.cloud.aiplatform.base:Delete MatchingEngineIndexEndpoint  backing LRO: projects/473197248954/locations/us-central1/operations/2627263449519882240


MatchingEngineIndexEndpoint deleted. . Resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


INFO:google.cloud.aiplatform.base:MatchingEngineIndexEndpoint deleted. . Resource name: projects/473197248954/locations/us-central1/indexEndpoints/5787503753171042304


In [73]:
# Delete indexes
tree_ah_index.delete()

Deleting MatchingEngineIndex : projects/473197248954/locations/us-central1/indexes/966681827047636992


INFO:google.cloud.aiplatform.base:Deleting MatchingEngineIndex : projects/473197248954/locations/us-central1/indexes/966681827047636992


Delete MatchingEngineIndex  backing LRO: projects/473197248954/locations/us-central1/indexes/966681827047636992/operations/8095759297054507008


INFO:google.cloud.aiplatform.base:Delete MatchingEngineIndex  backing LRO: projects/473197248954/locations/us-central1/indexes/966681827047636992/operations/8095759297054507008


MatchingEngineIndex deleted. . Resource name: projects/473197248954/locations/us-central1/indexes/966681827047636992


INFO:google.cloud.aiplatform.base:MatchingEngineIndex deleted. . Resource name: projects/473197248954/locations/us-central1/indexes/966681827047636992


In [103]:
import google.cloud.aiplatform_v1beta1 as aiplatform_v1beta1

In [101]:
import google.cloud.aiplatform.aiplatform_v1beta1 as aiplatform_v1beta1

ModuleNotFoundError: No module named 'google.cloud.aiplatform.aiplatform_v1beta1'